# Vectorisation

A requirement that we haven't yet addressed is the option to apply a land or ocean mask. To do this, we can use the corresponding land surface fraction file.

In [10]:
import iris
import numpy

In [11]:
access_sftlf_file = 'data/sftlf_fx_ACCESS1-3_historical_r0i0p0.nc'

In [12]:
sftlf_cube = iris.load_cube(access_sftlf_file, 'land_area_fraction')
print(sftlf_cube)

land_area_fraction / (%)            (latitude: 145; longitude: 192)
     Dimension coordinates:
          latitude                           x               -
          longitude                          -               x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_atmos_fx_ACCESS1-3_historical_r0i0p0.nc...
          branch_time: 90945.0
          cmor_version: 2.8.0
          contact: The ACCESS wiki: http://wiki.csiro.au/confluence/display/ACCESS/Home. Contact...
          creation_date: 2012-02-15T06:14:44Z
          experiment: historical
          experiment_id: historical
          forcing: GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2, N2O, CH4, CFC11, CFC12, CFC113,...
          frequency: fx
          history: 2012-02-15T06:14:43Z altered by CMOR: Converted units from '1' to '%'....
          initialization_method: 0
          institute_id: CSIRO-BOM
          institution: CS

/Users/damienirving/anaconda/envs/pyaos-lesson/lib/python3.6/site-packages/iris/fileformats/cf.py:798: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'sftlf'
  warnings.warn(message % (variable_name, nc_var_name))
/Users/damienirving/anaconda/envs/pyaos-lesson/lib/python3.6/site-packages/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


The data in a sftlf file assigns each grid cell a percentage value between 0% (no land) to 100% (all land).

In [13]:
print(sftlf_cube.data.max())
print(sftlf_cube.data.min())

100.0
0.0


To create a [numpy masked array](https://docs.scipy.org/doc/numpy/reference/maskedarray.html), we need to assign each grid cell a `True` (apply mask) or `False` (do not apply mask) value. For this example, we are going to define the ocean as any cell that is less than 50% land (and the land as any cell greater than 50%).

The most obvious solution to creating an ocean mask, for example, might then be to loop over each cell in the sftlf array. e.g.

```
nlats, nlons = sftlf_cube.data.shape
mask = numpy.zeros([nlats, nlons])
for y in range(nlats):
    for x in range(nlons):
        if sftlf_cube.data[y, x] < 50:
            mask[y, x] = True
        else:
            mask[y, x] = False
```

While this approach would technically work, the problem is that (a) the code is hard to read, and (b) in contrast to low level languages like Fortran and C, high level languages like Python and Matlab are built for usability (i.e. they make it easy to write concise, readable code) as opposed to speed. This particular array is so small that the looping isn't noticably slow, but in general looping over every data point in an array should be avoided.

Fortunately, there are lots of `numpy` functions that allow you to get around this problem by applying a particular operation to an entire array at once (which is known as a vectorised operation). The `numpy.where` function, for instance, allows us to make a true/false decision at each data point in the array and then perform a different action depending on the answer:

In [14]:
ocean_mask = numpy.where(sftlf_cube.data < 50, True, False)
land_mask = numpy.where(sftlf_cube.data > 50, True, False)

For a given iris cube (e.g. containing precipitation data from the ACCESS1-3 model), we could then convert the data type to a numpy masked array and apply our mask:

In [15]:
cube = iris.load_cube('data/pr_Amon_ACCESS1-3_historical_r1i1p1_200101-200512.nc', 'precipitation_flux')
print(type(cube.data))

<class 'numpy.ndarray'>


/Users/damienirving/anaconda/envs/pyaos-lesson/lib/python3.6/site-packages/iris/fileformats/cf.py:1143: IrisDeprecation: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warn_deprecated(msg)


In [16]:
cube.data = numpy.ma.asarray(cube.data)
cube.data.mask = ocean_mask
print(type(cube.data))

<class 'numpy.ma.core.MaskedArray'>


By printing the array we can see that some values are now masked:

In [17]:
print(cube.data[0, 100:110, 0:10])

[[2.4865681552910246e-05 1.7857040802482516e-05 1.4409809409698937e-05
  1.365557181998156e-05 9.77409854385769e-07 1.6724919760235935e-07 -- --
  -- --]
 [-- 3.080878741457127e-05 2.1845595256309025e-05 2.4051765649346635e-05
  1.7585027308086865e-05 1.6222168142121518e-06 -- -- -- --]
 [-- -- -- -- -- -- -- -- -- --]
 [2.005224087042734e-05 -- -- -- -- -- -- -- 3.166760870954022e-05
  1.2054460967192426e-05]
 [1.4890032616676763e-05 -- -- -- -- 2.3196560505311936e-05 -- --
  6.278240471147001e-05 2.7597512598731555e-05]
 [1.0251545063511003e-05 -- -- -- -- 3.0613329727202654e-05 -- --
  2.898933416872751e-05 --]
 [1.4132613614492584e-05 7.409330464724917e-06 -- -- -- -- --
  3.593934889067896e-05 -- --]
 [4.693903247243725e-05 3.182605723850429e-05 1.990660712181125e-05
  8.492984306940343e-06 -- -- 2.465165925968904e-05 -- --
  0.0001078539207810536]
 [6.071893949410878e-05 6.422175647458062e-05 5.667455116054043e-05
  2.7484224119689316e-05 9.198953193845227e-06 1.1014224583050236e

**Challenge:** 

1. Modify `plot_precipitation_climatology.py` so that the user can choose to apply a mask as follows (this should involve defining a new function called `apply_mask()`, in order to keep `main()` short and readable):
```
parser.add_argument("--mask", type=str, nargs=2, metavar=('SFTLF_FILE', 'REALM'), default=None,
                       help='Apply a land or ocean mask (specify the realm to mask)')
```

2. Test to see if your mask worked by plotting the ACCESS1-3 climatology for January:
```
$ python plot_precipitation_climatology.py data/pr_Amon_ACCESS1-3_historical_r1i1p1_200101-200512.nc Jan pr_Amon_ACCESS1-3_historical_r1i1p1_200101-200512-jan-clim_land-mask.png --mask data/sftlf_fx_ACCESS1-3_historical_r0i0p0.nc land
```
3. Commit the changes to git and then push to GitHub